<a href="https://colab.research.google.com/github/christinezuzart/DeepLearning/blob/master/NLPULMFitGPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import files

uploaded = files.upload()

In [1]:
import pandas as pd
import io

dfTrain = pd.read_csv(io.StringIO(uploaded['Train.csv'].decode('utf-8')))
dfTest = pd.read_csv(io.StringIO(uploaded['Test.csv'].decode('utf-8')))

train_texts = dfTrain.loc[ : , 'review'].values
train_labels = dfTrain.loc[ : , 'condition'].values

test_texts = dfTest.loc[ : , 'review'].values
test_labels = dfTest.loc[ : , 'condition'].values

col_names = ['labels','text']
df_train = pd.DataFrame({'text':train_texts, 'labels':train_labels}, columns=col_names)
df_test = pd.DataFrame({'text':test_texts, 'labels':test_labels}, columns=col_names)

df_train = df_train.dropna(axis=0)
df_test = df_test.dropna(axis=0)

unique_train_labels_list = df_train.labels.unique()
unique_test_labels_list = df_test.labels.unique()



NameError: ignored

In [0]:
df_test_new = pd.DataFrame(columns=['labels','text'])
for ind in df_test.index: 
  if df_test['labels'][ind] in unique_train_labels_list :
   new_row = {'labels': df_test['labels'][ind], 'text':df_test['text'][ind]}
   df_test_new = df_test_new.append(new_row, ignore_index=True)

print(df_test_new.head())

                         labels                                               text
0                    Depression  "I&#039;ve tried a few antidepressants over th...
1  Crohn's Disease, Maintenance  "My son has Crohn&#039;s disease and has done ...
2       Urinary Tract Infection                      "Quick reduction of symptoms"
3                   Weight Loss  "Contrave combines drugs that were used for al...
4                 Birth Control  "I have been on this birth control for one cyc...


In [0]:
a = df_test_new.labels.unique()
print('\n'.join(map(str, a)))

In [0]:
from fastai.text import *

# Language model data
data_lm = TextLMDataBunch.from_df('./', train_df=df_train, valid_df=df_test_new)

In [0]:
# Language model
lang_learner = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)
lang_learner.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,3.548549,3.368265,0.321313,11:21


In [0]:
lang_learner.unfreeze()
lang_learner.fit_one_cycle(1, 1e-3)
lang_learner.save_encoder('fine_enc')


epoch,train_loss,valid_loss,accuracy,time
0,3.157662,3.049905,0.364366,13:02


In [0]:
# Classifier model data
data_clas = TextClasDataBunch.from_df('./', train_df=df_train, valid_df=df_test_new, vocab=data_lm.train_ds.vocab, bs=32)




In [0]:
# Classifier
learn = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5).to_fp16()
learn.load_encoder('fine_enc')

In [0]:
learn.fit_one_cycle(1, 2e-2, moms=(0.8, 0.7))


epoch,train_loss,valid_loss,accuracy,time
0,2.099248,1.668530,0.622536,04:10


In [0]:
learn.unfreeze()
learn.fit_one_cycle(3, slice(1e-4, 1e-2))

epoch,train_loss,valid_loss,accuracy,time
0,1.510134,1.492698,0.666916,10:00
1,1.354954,1.266973,0.706280,10:38
2,1.129555,1.155791,0.734076,10:09


In [0]:
learn.export()

In [0]:
learn.reorder.plot_losses()

In [0]:
from fastai.vision import *

preds, y, losses = learn.get_preds(with_loss = 1)


In [0]:
interp = ClassificationInterpretation(learn, preds, y, losses)

interp.plot_confusion_matrix()

from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
model_save_name = 'export.pkl'
path = F"/content/gdrive/My Drive/{model_save_name}" 
torch.save(model.state_dict(), path)

In [0]:
model_save_name = 'export.pkl'
path = F"/content/gdrive/My Drive/{model_save_name}" 

saved_learner = load_learner(path)

saved_learner.predict("")